
# Overview
1.   Get passages from jnana (already extracted)
2.   Generate question and answer using chatGPT4
3.   Returns csv of generated question, passage and answer





# Installs

In [ ]:
!pip install openai

# Imports

In [ ]:
import os
import pandas as pd
import json
import time
import re

# openAI
from openai import OpenAI

# drive
from google.colab import drive

In [ ]:
# mount google drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/final_project_266'

 266_FP_SO_QExtract_gpt4.ipynb	 generated_q_a_df_25passages.csv
 batch_requests.jsonl		'generated_q_a_df (2).gsheet'
 extracted_passages_Jnana.csv	 generated_q_a_df.csv
'generated_q_a_df (1).gsheet'	 generated_q_a_df.gsheet


# Set up File Paths

In [ ]:
# jnana_path = '/content/drive/MyDrive/266 Final project/extracted_passages_Jnana.csv'
jnana_path = '/content/drive/MyDrive/Colab Notebooks/final_project_266/extracted_passages_Jnana.csv'

In [ ]:
jnana_passages_df = pd.read_csv(jnana_path)
jnana_passages_df = jnana_passages_df[3:] # remove publication information, only use passages with substance
jnana_passages_df = jnana_passages_df.reset_index(drop=True)
jnana_passages_df


,Passage
0,"When you turn over the pages, you are actually..."
1,"You have the unique chance of meeting Him, at ..."
2,"In the pages of the magazine, published with H..."
3,"N. KASTURI Editor, Sanathana Sarathi JNANA V A..."
4,It is only when full knowledge is won that one...
...,...
212,Samam - Control of the senses. Samsara - Chang...
213,"Sathwic/Sathwika - Pure, good and pious, calm,..."
214,Subhechchaa - The desire to procure one’s own ...
215,Thath Thwam Asi - “That thou art.” Thithiksha ...


In [ ]:
test_passage = jnana_passages_df['Passage'][3]
len(test_passage)

520

# Set up openAI


In [ ]:
# test key
client = OpenAI()


# test access
response = client.responses.create(
  model="gpt-4o-mini",
  input="write a haiku about ai",
  store=True,
)

print(response.output_text);

Silent thoughts arise,  
Lines of code weave dreams and truth,  
Mind of wire sings.  


# Test: Generate questions and answers

In [ ]:
# generate question and answer from this passage using chat gpt4

# estimated gpt usage
# --- question generation ---
# Prompt tokens: 153
# Completion tokens: 180
# Total tokens: 333
#
# --- answer generation ---
# Prompt tokens: 332
# Completion tokens: 261
# Total tokens: 593


def generate_question(passage):

  prompt = f"""
  You are a thoughtful educator. Read the passage below and come up with 5 unique thought provoking quesiton.
  {passage}
  """

  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7
  )

  # usage = response.usage
  # print("Prompt tokens:", usage.prompt_tokens)
  # print("Completion tokens:", usage.completion_tokens)
  # print("Total tokens:", usage.total_tokens)

  return response.choices[0].message.content.strip()

def generate_answer(passage, questions):

  prompt = f"""
  Answer the questions below using the passage provided in 2 sentences or less.
  Question: {questions}
  Passage: {passage}
  """

  response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.7
  )

  # usage = response.usage
  # print("Prompt tokens:", usage.prompt_tokens)
  # print("Completion tokens:", usage.completion_tokens)
  # print("Total tokens:", usage.total_tokens)

  return response.choices[0].message.content

def parse_questions(questions):
  question_list = []
  for q in questions.split('\n'):
    if q != '':
      # remove number, punctuation and white space
      q = q[3:]

      # append question
      question_list.append(q)
  return question_list

def parse_answers(answers):
  answer_list = []
  for a in answers.split('\n'):
    if a != '':
      # remove number, punctuation and white space
      a = a[3:]

      # append answer
      answer_list.append(a)
  return answer_list



### Test with the first viable passage

In [ ]:
test_q_a_df = pd.DataFrame(columns=[ 'Passage','Question','Answer'])
test_q_a_df

,Passage,Question,Answer


In [ ]:
q = generate_question(test_passage)
q

Prompt tokens: 153
Completion tokens: 180
Total tokens: 333


'1. What parallels can be drawn between the process of acquiring knowledge and the metaphor of removing the husk from rice, and how might this insight influence our approach to learning?\n\n2. In what ways does the concept of "uninterrupted Inquiry" challenge traditional methods of education, and how might this idea be implemented in modern teaching practices?\n\n3. How do the notions of birth and death contribute to our understanding of the self, and in what ways can inquiry into these transformations shape our perspective on life and existence?\n\n4. The passage mentions "Ignorance" as something that can be "melted away" through knowledge. How can this idea be applied to societal issues such as prejudice, misinformation, or cultural misunderstandings?\n\n5. Considering the reference to "the reality of the I," how might the exploration of individual identity through inquiry impact our relationships with others and our sense of community?'

In [ ]:
q_list = parse_questions(q)
print(q_list)
len(q_list)

['In what ways can the teachings of Bhagavan Sri Sathya Sai Baba be applied to modern-day challenges faced by individuals in their spiritual journeys?', 'How does the concept of surrendering one’s burdens to a higher power, as suggested in the passage, influence personal accountability and responsibility in one’s life?', 'What implications does the notion of an omnipresent guide have on our understanding of community and interconnectedness in addressing societal issues?', "How can the ideas of peace and perfection, as described by Bhagavan Sri Sathya Sai Baba, be interpreted in the context of mental health and well-being in today's fast-paced world?", 'In what ways might the call to “start on the spiritual pilgrimage” challenge conventional notions of success and fulfillment in contemporary society?']


5

In [ ]:
a = generate_answer(test_passage, q)
a


Prompt tokens: 332
Completion tokens: 261
Total tokens: 593


'1. The process of acquiring knowledge parallels removing the husk from rice as both involve peeling away layers that obscure the true essence; this insight suggests that a deeper, more persistent inquiry is essential for effective learning. Embracing this metaphor can encourage learners to engage more profoundly with their subjects to uncover underlying truths.\n\n2. "Uninterrupted Inquiry" challenges traditional methods of education by promoting continuous exploration rather than rote learning, fostering a more dynamic and engaged learning environment. This idea can be implemented in modern teaching practices through project-based learning, open-ended questions, and encouraging critical thinking.\n\n3. The notions of birth and death highlight the transient nature of the self, prompting inquiry into how these transformations shape our existence and identity. Exploring these concepts can lead to a deeper appreciation of life’s impermanence and the interconnectedness of all beings.\n\n4

In [ ]:
a_list = parse_answers(a)
a_list

['The teachings of Bhagavan Sri Sathya Sai Baba can be applied to modern challenges by encouraging individuals to seek inner peace and perfection through spiritual practice, thus navigating life’s difficulties with a greater sense of purpose. His guidance fosters resilience and clarity in the face of contemporary issues.',
 'Surrendering one’s burdens to a higher power promotes personal accountability by encouraging individuals to recognize their responsibilities while trusting that they are supported in their struggles. This balance allows for a healthier approach to challenges, fostering both personal growth and reliance on divine guidance.',
 'The notion of an omnipresent guide highlights the importance of community and interconnectedness, suggesting that collective efforts are essential in addressing societal issues. It reinforces the idea that individual well-being is linked to the well-being of others, promoting a spirit of cooperation.',
 'The ideas of peace and perfection can b

### Sample questions and answer

#### Passage
When you turn over the pages, you are actually sitting at the Feet of Bhagavan Sri Sathya Sai Baba, the Avatar of the age, come in answer to the prayersof all Sadhus and Sadhakas to guide them and grant them peace and perfection. “Place all your burdens on Me,” He says. “Start on the spiritual pilgrimage this very Day,”He exhorts. “Why fear, when I am here?” He asks. His Grace is Omnipresent. His Miraculous Powers proclaim His Omnipotence. His wisdom, His analysis of the ills ofhumanity and His Prescription for their cure reveal His Omniscience.

#### Generated questions

1. How do the teachings of Bhagavan Sri Sathya Sai Baba inspire individuals to confront their personal challenges and burdens in a spiritual context?

2. In what ways can the concept of an "Avatar" serve as a source of comfort and motivation for individuals on their spiritual journeys?

3. What does it mean to "place all your burdens" on a divine figure, and how might this practice influence one's mental and emotional well-being?

4. How does the idea of an omnipresent, omnipotent, and omniscient guide shape our understanding of spirituality and personal responsibility in addressing the ills of humanity?

5. In what ways can we interpret the call to "start on the spiritual pilgrimage this very day" in the context of modern life, where distractions and challenges abound?

#### Generated Answers
1. The teachings of Bhagavan Sri Sathya Sai Baba encourage individuals to confront their personal challenges by reminding them of his divine presence and guidance, which fosters a sense of peace and resolution.

2. The concept of an "Avatar" serves as a source of comfort and motivation by embodying divine qualities that inspire individuals to trust in a higher power during their spiritual journeys.

3. To "place all your burdens" on a divine figure means to surrender one's worries and struggles to a higher power, which can significantly enhance one's mental and emotional well-being by alleviating stress and promoting inner peace.

4. The idea of an omnipresent, omnipotent, and omniscient guide shapes our understanding of spirituality by emphasizing the importance of divine support while also highlighting our personal responsibility in addressing the ills of humanity.

5. The call to "start on the spiritual pilgrimage this very day" encourages individuals to prioritize their spiritual growth amidst modern distractions and challenges, emphasizing that every moment is an opportunity for personal transformation.

In [ ]:
# add questions and answers to data frame with the associated passage
q_per_passage = 5

q_list = parse_questions(q)
a_list = parse_answers(a)

for i in range(q_per_passage):
    test_q_a_df.at[i, 'Passage'] = test_passage
    test_q_a_df.at[i, 'Question'] = q_list[i]
    test_q_a_df.at[i, 'Answer'] = a_list[i]

# test output
test_q_a_df


,Passage,Question,Answer
0,"When you turn over the pages, you are actually...",In what ways can the teachings of Bhagavan Sri...,The teachings of Bhagavan Sri Sathya Sai Baba ...
1,"When you turn over the pages, you are actually...",How does the concept of surrendering one’s bur...,Surrendering one’s burdens to a higher power p...
2,"When you turn over the pages, you are actually...",What implications does the notion of an omnipr...,The notion of an omnipresent guide highlights ...
3,"When you turn over the pages, you are actually...","How can the ideas of peace and perfection, as ...",The ideas of peace and perfection can be inter...
4,"When you turn over the pages, you are actually...",In what ways might the call to “start on the s...,"The call to ""start on the spiritual pilgrimage..."


# Generate all Questions and Answers

In [ ]:
generated_q_a_df = pd.DataFrame(columns=[ 'Passage','Question','Answer'])

#### Syncronous API implementation -- do not use --

---



In [ ]:
import time
from openai import RateLimitError

q_per_passage = 5
generated_q_a_df = pd.DataFrame(columns=[ 'Passage','Question','Answer'])

for p in range(len(jnana_passages_df['Passage'])):
# for p in range(2):
  print(p)
  passage = jnana_passages_df['Passage'][p]

  try:
    gen_q = generate_question(passage)
    gen_a = generate_answer(passage, gen_q) # Use gen_q here, not the old q
  except RateLimitError as e:
    print(f"Rate limit exceeded: {e}")
    print("Waiting for 10 minutes before retrying...")
    time.sleep(600)  # Wait for 10 minutes
    # Retry the current passage
    p -= 1
    continue


  q_list = parse_questions(gen_q)
  a_list = parse_answers(gen_a)

  for j in range(q_per_passage):
    if j < len(q_list) and j < len(a_list): # Add a check to prevent index errors
        generated_q_a_df.at[j+(q_per_passage*p), 'Passage'] = passage
        generated_q_a_df.at[j+(q_per_passage*p), 'Question'] = q_list[j]
        generated_q_a_df.at[j+(q_per_passage*p), 'Answer'] = a_list[j]
    else:
        print(f"Warning: Skipping Q/A pair {j+1} for passage {p} due to insufficient generated questions or answers.")

0


KeyboardInterrupt: 

In [ ]:
generated_q_a_df

,Passage,Question,Answer


## Batch API -- use this one ---

##### Prepare batch requests


In [ ]:
test_jnana_df = jnana_passages_df[0:2]
test_jnana_df

,Passage
0,"When you turn over the pages, you are actually..."
1,"You have the unique chance of meeting Him, at ..."


In [ ]:
jnana_passages_25_df = jnana_passages_df[0:25]
jnana_passages_25_df

,Passage
0,"When you turn over the pages, you are actually..."
1,"You have the unique chance of meeting Him, at ..."
2,"In the pages of the magazine, published with H..."
3,"N. KASTURI Editor, Sanathana Sarathi JNANA V A..."
4,It is only when full knowledge is won that one...
5,Until one gets firmly fixed in the path that g...
6,When Ignorance and its concomitant Delusion di...
7,Where did that light emanate from? From the go...
8,So one has to conclude that the splendour came...
9,"To realise this Atma, this Jnanaswarupa (Embod..."


In [ ]:
jnana_passages_df_remaining = jnana_passages_df[25:]
jnana_passages_df_remaining

,Passage
25,"Therefore, the Jnani who has had a vision of t..."
26,"By this means, every trace of “in-tention” and..."
27,The constant contemplation of the Atma and its...
28,The mind can be trained by following the metho...
29,"Very often with the progress of Dhyana, new de..."
...,...
212,Samam - Control of the senses. Samsara - Chang...
213,"Sathwic/Sathwika - Pure, good and pious, calm,..."
214,Subhechchaa - The desire to procure one’s own ...
215,Thath Thwam Asi - “That thou art.” Thithiksha ...


In [ ]:
tasks = []

# for p_idx, passage in enumerate(jnana_passages_df['Passage']):
for p_idx, passage in enumerate(jnana_passages_df_remaining["Passage"]):
      custom_id = f"{p_idx}"
      prompt = (
          f"Passage:\n{passage}\n\n"
          "Generate exactly five thoughtful questions based on the passage "
          "and its corresponding answer in 1-2 sentences. Format:\nQuestion: …\nAnswer: …"
      )
      task = {
          "custom_id": custom_id,
          "method": "POST",
          "url": "/v1/chat/completions",
          "body": {
              "model": "gpt-4o-mini",  # or "gpt-3.5-turbo"
              "temperature": 0.7,
              "max_tokens": 500,
              "messages": [
                  {"role": "system", "content": "You are a thoughtful educator."},
                  {"role": "user", "content": prompt}
              ],
          }
      }
      tasks.append(task)
# batch_request_filepath = "/content/drive/MyDrive/266 Final project/batch_requests.jsonl"
batch_request_filepath = "/content/drive/MyDrive/Colab Notebooks/final_project_266/batch_requests.jsonl"
with open(batch_request_filepath, "w") as f:
    for t in tasks:
        f.write(json.dumps(t) + "\n")

### Upload and start batch job

In [ ]:
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

In [ ]:
batch_file = client.files.create(file=open(batch_request_filepath, "rb"), purpose="batch")
batch_job = client.batches.create(input_file_id=batch_file.id,
                                  endpoint="/v1/chat/completions",
                                  completion_window="24h")

##### Poll for completion and Retrieve Results

In [ ]:
while True:
    job = client.batches.retrieve(batch_job.id)
    if job.status != "completed":
        print("Batch status:", job.status)
        time.sleep(30)
    else:
        print("Batch completed!")
        break

output = client.files.content(job.output_file_id).content.decode("utf-8")
results = [json.loads(line) for line in output.strip().split("\n")]

Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status: in_progress
Batch status

##### Parse and Populate Dataframe


In [ ]:
def parse_qa_text(text: str):
    """
    Parses QA pairs from a text block with optional answers.
    Returns a list of dicts: [{'question': ..., 'answer': ...}, ...]
    If an answer is missing, 'answer' will be set to None.
    """
    # Regex: capture 'Question:', then optional 'Answer:' text before next Question or end
    pattern = re.compile(
        r"Question:\s*(?P<q>.*?)(?:\s*Answer:\s*(?P<a>.*?))?(?=(?:\s*Question:|$))",
        re.DOTALL
    )
    qa_pairs = []
    for m in pattern.finditer(text):
        q = m.group("q").strip()
        a = m.group("a")
        answer = a.strip() if a and a.strip() else None
        qa_pairs.append({'question': q, 'answer': answer})
    return qa_pairs


In [ ]:
results[0]

body = results[0]["response"]["body"]
content = body["choices"][0]["message"]["content"]

print(content)

Question: What is the significance of the Jnani's vision of the Atma in relation to suffering?  
Answer: The Jnani, having realized the Atma within, transcends sorrow as they recognize the eternal essence shared by all living beings, which fosters a sense of unity and peace.

Question: How does the passage describe the Atma's presence in living beings?  
Answer: The passage states that the Atma exists in all living things, from the smallest ant to the largest elephant, emphasizing its fundamental role in sustaining the entire world.

Question: What is the role of the Sadhaka in the context of this passage?  
Answer: The Sadhaka, or spiritual aspirant, is encouraged to shift focus from the external world to the inner self, seeking to understand and analyze the workings of their own mind.

Question: What does the passage suggest about the relationship between the mind's modifications and the Atma?  
Answer: The passage implies that the agitations and changes in the mind stem from a lack 

In [ ]:
results[1]["custom_id"]

'1'

In [ ]:
generated_q_a_df = pd.DataFrame(columns=['Passage','Question','Answer'])
n = 0
q_per_passage = 5

for entry in results:
    cid = entry["custom_id"]
    # p_idx, j = map(int, cid.split("_"))
    # p_idx = map(int, cid.split("_"))
    p_idx = int(cid)

    body = entry["response"]["body"]
    content = body["choices"][0]["message"]["content"]

    qa_list = parse_qa_text(content)
    for n in range(q_per_passage):
      if n < len(qa_list):
        generated_q_a_df.at[n+(q_per_passage*p_idx), 'Passage'] = jnana_passages_df['Passage'][p_idx]
        generated_q_a_df.at[n+(q_per_passage*p_idx), 'Question'] = qa_list[n]['question']
        generated_q_a_df.at[n+(q_per_passage*p_idx), 'Answer'] = qa_list[n]['answer']

        # generated_q_a_df.at[n+(q_per_passage*n), 'Passage'] = jnana_passages_df['Passage'][]
        # generated_q_a_df.at[n+(q_per_passage*n), 'Question'] = qa_list[n]['question']
        # generated_q_a_df.at[n+(q_per_passage*n), 'Answer'] = qa_list[n]['answer']

      else:
        print(f"Warning: Skipping Q/A pair {n+1} for passage {p_idx} due to insufficient generated questions or answers.")

    n += 1

In [ ]:
generated_q_a_df

,Passage,Question,Answer
0,"When you turn over the pages, you are actually...",What is the significance of the Jnani's vision...,"The Jnani, having realized the Atma within, tr..."
1,"When you turn over the pages, you are actually...",How does the passage describe the Atma's prese...,The passage states that the Atma exists in all...
2,"When you turn over the pages, you are actually...",What is the role of the Sadhaka in the context...,"The Sadhaka, or spiritual aspirant, is encoura..."
3,"When you turn over the pages, you are actually...",What does the passage suggest about the relati...,The passage implies that the agitations and ch...
4,"When you turn over the pages, you are actually...",What process is recommended for the Sadhaka to...,The Sadhaka is advised to engage in self-analy...
...,...,...,...
955,"Of course, he who has understood the scripture...","What does the term ""Vikalpas"" refer to in the ...","""Vikalpas"" refers to mental agitations, highli..."
956,"Of course, he who has understood the scripture...","How does ""Viveka"" contribute to one's understa...","""Viveka"" is the faculty of discrimination that..."
957,"Of course, he who has understood the scripture...","What is the significance of ""Viratpurusha"" in ...","""Viratpurusha,"" or the Cosmic Person, symboliz..."
958,"Of course, he who has understood the scripture...","In what way does ""Vikshepa"" relate to ignorance?","""Vikshepa"" denotes a state of waywardness wher..."


# Convert generated questions and answers data frame to csv


In [ ]:
# test to csv
# generated_q_a_df.to_csv('/content/drive/MyDrive/266 Final project/generated_q_a_df.csv', index=False)
generated_q_a_df.to_csv('/content/drive/MyDrive/Colab Notebooks/final_project_266/generated_q_a_df.csv', index=False)

In [ ]:
# File paths for the two CSV files
file_path_1 = '/content/drive/MyDrive/Colab Notebooks/final_project_266/generated_q_a_df.csv'
file_path_2 = '/content/drive/MyDrive/Colab Notebooks/final_project_266/generated_q_a_df_25passages.csv'

# Read the CSV files into pandas DataFrames
df1 = pd.read_csv(file_path_1)
df2 = pd.read_csv(file_path_2)

# Combine the two DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# Display the combined DataFrame
display(combined_df)

,Passage,Question,Answer
0,"When you turn over the pages, you are actually...",What is the significance of the Jnani's vision...,"The Jnani, having realized the Atma within, tr..."
1,"When you turn over the pages, you are actually...",How does the passage describe the Atma's prese...,The passage states that the Atma exists in all...
2,"When you turn over the pages, you are actually...",What is the role of the Sadhaka in the context...,"The Sadhaka, or spiritual aspirant, is encoura..."
3,"When you turn over the pages, you are actually...",What does the passage suggest about the relati...,The passage implies that the agitations and ch...
4,"When you turn over the pages, you are actually...",What process is recommended for the Sadhaka to...,The Sadhaka is advised to engage in self-analy...
...,...,...,...
1080,Hewill not mistake the rope for the snake. He ...,"What does the reference to the ""rope"" and ""sna...","The ""rope"" and ""snake"" symbolize the distincti..."
1081,Hewill not mistake the rope for the snake. He ...,"What is the significance of the term ""Abhasa A...","""Abhasa Avaranam"" refers to the veil of illusi..."
1082,Hewill not mistake the rope for the snake. He ...,How does the passage describe the nature of Pa...,"The passage suggests that Paramatma, or the su..."
1083,Hewill not mistake the rope for the snake. He ...,"What does the statement about ""effulgence of B...",It implies that individuals often fail to see ...


In [ ]:

combined_df.to_csv('/content/drive/MyDrive/Colab Notebooks/final_project_266/full_generated_q_a_jnana.csv', index=False)